# Défini data engineer codoc

In [1]:
# Importation des librairies
import pandas as pd
import re
import os
import pdfplumber
from sqlalchemy import create_engine
import docx

# Exercice 1 : Chargement des patients
Dans cet exercice, le contenu du fichier `export_patient.xlsx` sera standardisé et ingéré dans les tables DWH_PATIENT et DWH_PATIENT_IPPHIST. 

In [2]:
# Chargement du fichier export_patient.xlsx
export_patient = pd.read_excel('fichiers source/export_patient.xlsx')
export_patient.head()

,NOM,PRENOM,DATE_NAISSANCE,SEXE,NOM_JEUNE_FILLE,HOSPITAL_PATIENT_ID,ADRESSE,TEL,CP,VILLE,PAYS,DATE_MORT
0,Varner,Daniel,25/07/1942,M,NaN,15115800,1324 Cste Joyeuse,819-226-5974,H0H 0H0,Aston Junction,Canada,NaN
1,Metcalf,John,15/06/1936,M,NaN,15120924,Grsnvangen 15,53-60-89-59,6600,Vejen,Denmark,NaN
2,Burns,Christopher,08/06/1985,M,NaN,15126048,Ssp utca 13.,(27) 756-777,2164,Vschartysn,Hungary,NaN
3,Johnson,Thomas,31/10/1940,M,NaN,15131172,Sonnenbergstr 44,021 349 41 88,1329,Bretonnisres,Switzerland,NaN
4,Howell,Margaret,07/08/2012,F,NaN,15136296,Quevedo 75,770 436 301,15360,Cariso,Spain,NaN


In [3]:
export_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4828 entries, 0 to 4827
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NOM                  4828 non-null   object 
 1   PRENOM               4828 non-null   object 
 2   DATE_NAISSANCE       4828 non-null   object 
 3   SEXE                 4828 non-null   object 
 4   NOM_JEUNE_FILLE      0 non-null      float64
 5   HOSPITAL_PATIENT_ID  4828 non-null   int64  
 6   ADRESSE              4828 non-null   object 
 7   TEL                  4828 non-null   object 
 8   CP                   4828 non-null   object 
 9   VILLE                4828 non-null   object 
 10  PAYS                 4826 non-null   object 
 11  DATE_MORT            581 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 452.8+ KB


In [4]:
type(export_patient['DATE_MORT'].unique()[0])

float

In [5]:
export_patient.nunique()

NOM                    2677
PRENOM                 1273
DATE_NAISSANCE         4506
SEXE                      2
NOM_JEUNE_FILLE           0
HOSPITAL_PATIENT_ID    4828
ADRESSE                4789
TEL                    4826
CP                     3963
VILLE                  3587
PAYS                     31
DATE_MORT               555
dtype: int64

In [6]:
# Connexion à la base de données
engine = create_engine('sqlite:///drwh.db')
conn=engine.connect()

In [7]:
# Liste des tables de la base de données
pd.read_sql_query('''select * from sqlite_master''', conn)

,type,name,tbl_name,rootpage,sql
0,table,DWH_PATIENT,DWH_PATIENT,2,CREATE TABLE DWH_PATIENT\n(\n PATIENT_NUM ...
1,table,DWH_PATIENT_IPPHIST,DWH_PATIENT_IPPHIST,3,CREATE TABLE DWH_PATIENT_IPPHIST\n(\n PATIENT...
2,table,DWH_DOCUMENT,DWH_DOCUMENT,4,CREATE TABLE DWH_DOCUMENT\n(\n DOCUMENT_NUM ...


In [8]:
conn.execute('''delete from dwh_patient''')
conn.execute('''delete from dwh_patient_ipphist''')
conn.execute('''delete from dwh_document''')


On souhaite savoir si un patient peut être présent plusieurs fois dans le fichier export_patient. 

On admet dans un premier temps qu'un patient est identifié par son nom, son prénom et sa date de naissance.

In [9]:
export_patient.groupby(['NOM','PRENOM','DATE_NAISSANCE']).size().reset_index().sort_values(by=0, ascending=False)

,NOM,PRENOM,DATE_NAISSANCE,0
2506,Larsen,Debbie,10/10/1920,2
3098,Muncie,Samuel,06/10/1934,2
0,Abarca,Krista,14/07/1986,1
3216,Oliver,Duane,28/02/2001,1
3222,Olson,Daniel,27/01/2015,1
...,...,...,...,...
1608,Glass,Vanessa,17/03/1918,1
1607,Glass,Ryan,08/04/1976,1
1606,Givan,Christopher,16/03/1922,1
1605,Giron,James,20/04/1953,1


Debbie Larsen et Samuel Muncie sont à deux endroits dans le fichier, on regarde de plus près les données pour comprendre s'il s'agit ou non des mêmes patients

In [10]:
export_patient[(export_patient['NOM']=='Muncie') & (export_patient['PRENOM']=='Samuel')]

,NOM,PRENOM,DATE_NAISSANCE,SEXE,NOM_JEUNE_FILLE,HOSPITAL_PATIENT_ID,ADRESSE,TEL,CP,VILLE,PAYS,DATE_MORT
1987,Muncie,Samuel,06/10/1934,M,NaN,6604836,Leandro Gomez 4417,9422 6980,97007,Centenario,Uruguay,NaN
4827,Muncie,Samuel,06/10/1934,M,NaN,302316,Leandro Gomez 4417,9422 6980,97007,Centenario,NaN,NaN


In [11]:
export_patient[(export_patient['NOM']=='Larsen') & (export_patient['PRENOM']=='Debbie')]

,NOM,PRENOM,DATE_NAISSANCE,SEXE,NOM_JEUNE_FILLE,HOSPITAL_PATIENT_ID,ADRESSE,TEL,CP,VILLE,PAYS,DATE_MORT
1843,Larsen,Debbie,10/10/1920,F,NaN,5866980,R Muro Bacalhoeiros 90,21 229 565 4006,4300-396,Porto,Portugal,NaN
4826,Larsen,Debbie,10/10/1920,F,NaN,307440,R Muro Bacalhoeiros 90,21 229 565 4006,4300-396,Porto,NaN,NaN


Dans les deux cas, les deux lignes semblent concerner le même patient : même sexe, même adresse, même numéro de téléphone, même code postal et même ville. 

Pour ces deux patients, on choisit la ligne où le pays est renseigné.

In [12]:
export_patient[export_patient['PAYS'].isna()]

,NOM,PRENOM,DATE_NAISSANCE,SEXE,NOM_JEUNE_FILLE,HOSPITAL_PATIENT_ID,ADRESSE,TEL,CP,VILLE,PAYS,DATE_MORT
4826,Larsen,Debbie,10/10/1920,F,NaN,307440,R Muro Bacalhoeiros 90,21 229 565 4006,4300-396,Porto,NaN,NaN
4827,Muncie,Samuel,06/10/1934,M,NaN,302316,Leandro Gomez 4417,9422 6980,97007,Centenario,NaN,NaN


Seules ces 2 lignes sont concernées par une valeur manquante dans la colonne PAYS. 

### Table DWH_PATIENT

In [13]:
# Récupération des colonnes d'intéret
dwh_patient = export_patient[['NOM', 'PRENOM','DATE_NAISSANCE','SEXE','NOM_JEUNE_FILLE','ADRESSE','TEL','CP', 'VILLE', 'DATE_MORT', 'PAYS']]

# Renommage des colonnes
dwh_patient = dwh_patient.rename(columns={
    'NOM' : 'LASTNAME', \
    'PRENOM' : 'FIRSTNAME', \
    'DATE_NAISSANCE' : 'BIRTH_DATE', \
    'SEXE' : 'SEX', \
    'NOM_JEUNE_FILLE' : 'MAIDEN_NAME', \
    'ADRESSE' : 'RESIDENCE_ADDRESS', \
    'TEL' : 'PHONE_NUMBER', \
    'CP' : 'ZIP_CODE', \
    'VILLE' : 'RESIDENCE_CITY', \
    'DATE_MORT' : 'DEATH_DATE', \
    'PAYS' : 'RESIDENCE_COUNTRY'
})

# Ajout DEATH_CODE
dwh_patient['DEATH_CODE'] = [1 if str(death_date)[2] == '/' else 0 for death_date in dwh_patient['DEATH_DATE']]

# On supprime les lignes pour lesquelles le pays de résidence n'est pas renseigné pour supprimer les doublons de Debbie Larsen et Samuel Muncie.
dwh_patient = dwh_patient[~dwh_patient['RESIDENCE_COUNTRY'].isna()]


dwh_patient.head()

,LASTNAME,FIRSTNAME,BIRTH_DATE,SEX,MAIDEN_NAME,RESIDENCE_ADDRESS,PHONE_NUMBER,ZIP_CODE,RESIDENCE_CITY,DEATH_DATE,RESIDENCE_COUNTRY,DEATH_CODE
0,Varner,Daniel,25/07/1942,M,NaN,1324 Cste Joyeuse,819-226-5974,H0H 0H0,Aston Junction,NaN,Canada,0
1,Metcalf,John,15/06/1936,M,NaN,Grsnvangen 15,53-60-89-59,6600,Vejen,NaN,Denmark,0
2,Burns,Christopher,08/06/1985,M,NaN,Ssp utca 13.,(27) 756-777,2164,Vschartysn,NaN,Hungary,0
3,Johnson,Thomas,31/10/1940,M,NaN,Sonnenbergstr 44,021 349 41 88,1329,Bretonnisres,NaN,Switzerland,0
4,Howell,Margaret,07/08/2012,F,NaN,Quevedo 75,770 436 301,15360,Cariso,NaN,Spain,0


In [14]:
# Ingestion du dataframe dans la table dwh_patient
dwh_patient.to_sql('dwh_patient', \
                   con = conn, \
                   if_exists='append',
                   index=False)

4826

In [15]:
# Lecture de la table
pd.read_sql_query('''select * from dwh_patient''', conn)

,PATIENT_NUM,LASTNAME,FIRSTNAME,BIRTH_DATE,SEX,MAIDEN_NAME,RESIDENCE_ADDRESS,PHONE_NUMBER,ZIP_CODE,RESIDENCE_CITY,...,RESIDENCE_LATITUDE,RESIDENCE_LONGITUDE,DEATH_CODE,UPDATE_DATE,BIRTH_COUNTRY,BIRTH_CITY,BIRTH_ZIP_CODE,BIRTH_LATITUDE,BIRTH_LONGITUDE,UPLOAD_ID
0,1,Varner,Daniel,25/07/1942,M,None,1324 Cste Joyeuse,819-226-5974,H0H 0H0,Aston Junction,...,None,None,0,None,None,None,None,None,None,None
1,2,Metcalf,John,15/06/1936,M,None,Grsnvangen 15,53-60-89-59,6600,Vejen,...,None,None,0,None,None,None,None,None,None,None
2,3,Burns,Christopher,08/06/1985,M,None,Ssp utca 13.,(27) 756-777,2164,Vschartysn,...,None,None,0,None,None,None,None,None,None,None
3,4,Johnson,Thomas,31/10/1940,M,None,Sonnenbergstr 44,021 349 41 88,1329,Bretonnisres,...,None,None,0,None,None,None,None,None,None,None
4,5,Howell,Margaret,07/08/2012,F,None,Quevedo 75,770 436 301,15360,Cariso,...,None,None,0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4821,4822,Fowler,Steven,13/06/1972,M,None,Lidicks 1490,466 936 404,570 01,Litomysl,...,None,None,0,None,None,None,None,None,None,None
4822,4823,Roman,Martine,24/08/1926,F,None,Scheidweg 81,062 855 66 31,5317,Hagenfirst,...,None,None,0,None,None,None,None,None,None,None
4823,4824,Lacasse,Theresa,08/01/1932,F,None,416 Gore Street,713-605-9869,77478,Sugar Land,...,None,None,0,None,None,None,None,None,None,None
4824,4825,Egan,Nicholas,31/08/2002,M,None,Rue du Centre 230,0496 80 77 81,7050,Masnuy-Saint-Pierre,...,None,None,0,None,None,None,None,None,None,None


PATIENT_NUM est créé automatiquement, il s'agit d'un entier incrémental. 

### DWH_PATIENT_IPPHIST

In [16]:
# Récupération des colonnes d'intéret
dwh_patient_ipphist = export_patient[['HOSPITAL_PATIENT_ID', 'NOM', 'PRENOM', 'DATE_NAISSANCE']]

# Renommage des colonnes
dwh_patient_ipphist = dwh_patient_ipphist.rename(columns={
    'NOM' : 'LASTNAME', \
    'PRENOM' : 'FIRSTNAME', \
    'DATE_NAISSANCE' : 'BIRTH_DATE'
})
dwh_patient_ipphist.head()

,HOSPITAL_PATIENT_ID,LASTNAME,FIRSTNAME,BIRTH_DATE
0,15115800,Varner,Daniel,25/07/1942
1,15120924,Metcalf,John,15/06/1936
2,15126048,Burns,Christopher,08/06/1985
3,15131172,Johnson,Thomas,31/10/1940
4,15136296,Howell,Margaret,07/08/2012


Cette table ne peut être ingérée en temps que DWH_PATIENT_IPP_HIST.
Elle sera ingérée en table intermédiaire, puis jointe avec la table DWH_PATIENT pour récupérer le PATIENT_NUM associé au HOSPITAL_PATIENT_ID.

In [17]:
# Ingestion de la table en temps que intermédiaire
dwh_patient_ipphist.to_sql('dwh_patient_ipphist_intermediaire', \
                           con = conn, \
                           if_exists='append', \
                        index=False)

4828

In [18]:
# Jointure de la table intermédiaire, puis ingestion dans la table DWH_PATIENT_IPPHIST
query = '''
INSERT INTO DWH_PATIENT_IPPHIST (PATIENT_NUM, HOSPITAL_PATIENT_ID)
SELECT 
    dwh_patient.PATIENT_NUM,
    dwh_patient_ipphist_intermediaire.HOSPITAL_PATIENT_ID
FROM dwh_patient_ipphist_intermediaire
JOIN dwh_patient USING(LASTNAME,FIRSTNAME,BIRTH_DATE)'''

conn.execute(query)


In [19]:
# Lecture des données
pd.read_sql_query('select * from dwh_patient_ipphist', conn)

,PATIENT_NUM,HOSPITAL_PATIENT_ID,ORIGIN_PATIENT_ID,MASTER_PATIENT_ID,UPLOAD_ID
0,1,15115800,None,None,None
1,2,15120924,None,None,None
2,3,15126048,None,None,None
3,4,15131172,None,None,None
4,5,15136296,None,None,None
...,...,...,...,...,...
4823,4824,24728424,None,None,None
4824,4825,24733548,None,None,None
4825,4826,24738672,None,None,None
4826,1844,307440,None,None,None


In [20]:
# Suppression de la table intermédiaire
conn.execute('''drop table dwh_patient_ipphist_intermediaire''')

# Exercice 2 : Chargement des documents
Dans cet exercice, le contenu des compte-rendus du dossier `fichiers source`sera ingéré dans la table DWH_DOCUMENT

In [21]:
# Quelques fonctions pour commencer

class File():
    '''Cette classe définie un fichier de compte-rendu .pdf ou .docx à traiter'''

    def __init__(self,file_path):
        self.file_path = file_path

    def extract_text_from_docx(self):
        '''Cette fonction extrait le texte d'un document .docx'''
        text = ""
        doc = docx.Document(self.file_path)
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"

        return text

    def extract_text_from_pdf(self):
        '''Cette fonction extrait le texte d'un document .pdf'''
        text=""
        with pdfplumber.open(self.file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text()
        return text

    def extract_text_from_file(self):
        '''Cette fonction extrait le texte d'un document .docx ou .pdf '''

        extension = os.path.basename(self.file_path).split('.')[1]

        if extension == 'pdf':
            text = self.extract_text_from_pdf()
                    
        if extension == 'docx':
            text = self.extract_text_from_docx()
            
        return(text)


    def extract_ipp_and_iddocument(self):
        '''Cette fonction renvoie l'ipp et l'iddocument d'un fichier'''

        basename_file = os.path.basename(self.file_path).split('.')[0]

        # Découpage du nom du fichier en 2 parties : IPP et IDDOCUMENT
        basename_file_split = basename_file.split('_')

        return(basename_file_split)


class Text():
    '''Cette classe défini un texte à analyser'''

    def __init__(self, text):
        self.text = text

    def find_author(self):
        '''Renvoie tous les auteurs probables d'un texte
        Un auteur est défini comme :
        - Un groupe de mot présent dans les 100 derniers caractères du texte
        - commençant par :
            - Dr
            - DR
            - Dr.
            '''
        matches = re.findall(r"(Dr\s+\w+(?:\s+\w+)*)|(Dr.\s+\w+(?:\s+\w+)*)|(DR\s+\w+(?:\s+\w+)*)", self.text[-100:])
        return(matches)

    def find_date(self):
        '''Renvoie toutes les dates possibles
        La date d'un document est définie comme une date après les mots : 
        - consultation du
        - hospitalisation du
        - Paris le
            '''
        matches = re.findall(r"hospitalisation\sdu\s+(\d{2}/\d{2}/\d{4})|consultation\sdu\s+(\d{2}/\d{2}/\d{4})|Paris\sle\s+(\d{2}/\d{2}/\d{4})", self.text)
        return(matches)


    def find_first_pattern(self, type):
        if type == 'author':
            matches = self.find_author()
        if type == 'date':
            matches = self.find_date()

        return([x for x in (matches[0]) if x != ''][0])



In [22]:

# Dossier contenant les compte-rendus
source_dir = 'fichiers source'

# Liste des fichiers du dossier
list_files = [source_dir + "/" + file for file in os.listdir(source_dir)]

dwh_document = pd.DataFrame(columns=['IPP','IDDOCUMENT','TEXT','AUTEUR'])

for file_path in list_files:

    # Si le fichier est un compte-rendu (commence par 8 chiffres)
    if re.match(r'^\d{8}', os.path.basename(file_path).split('.')[0]):

        file = File(file_path)

        # Extraction de l'IPP et de l'IDDOCUMENT du nom du fichier
        ipp_and_iddocument = file.extract_ipp_and_iddocument()

        dwh_document_file = pd.DataFrame({'IPP' : [ipp_and_iddocument[0]], \
                                    'IDDOCUMENT' : [ipp_and_iddocument[1]]})

        # Extraction du nom du fichier
        text = file.extract_text_from_file()

        dwh_document_file['TEXT'] = [text]

        text = Text(text)

        # Extraction de l'auteur du fichier
        try :
            dwh_document_file['AUTEUR'] = text.find_first_pattern('author')
        except :
            print("No autor found in file", file)

        # Extraction de la date du fichier
        try :
            dwh_document_file['DATE'] = text.find_first_pattern('date')
        except :
            print("No date found in file", file)
        
        dwh_document = pd.concat([dwh_document, dwh_document_file])

dwh_document = dwh_document.reset_index(drop=True)

dwh_document

No autor found in file <__main__.File object at 0x000002112858CE10>


,IPP,IDDOCUMENT,TEXT,AUTEUR,DATE
0,00230580,321654,Patient : Michelle Personne\nDate de naissance...,Dr Mamy Nova,12/05/2018
1,12558924,090879,Claus Norman Née le 06/10/1999\nIPP 12558924\n...,Dr Hassan,17/10/2011
2,12856116,789456,Alexander Betty Né le 09/11/1990\nIPP 12856116...,Dr. Abricot,26/10/2018
3,12881736,102323,"Paris le 10/02/2015,\nOrdonnance\nIn French po...",Dr Bilingue,10/02/2015
4,12907356,009988,Sabrina Franck Né le 09/07/1980\nIPP 12907356\...,DR Pomme,29/10/2018
5,12943224,976094,"Paris le 12/03/2016,\nOrdonnance\nAvant de voi...",Dr Hu Love,12/03/2016
6,12979092,645329,"Paris le 02/02/2002,\nOrdonnance\nAvant de voi...",Dr Mozza,02/02/2002
7,15156792,111222,Fred Taylor Né le 09/10/1976\nIPP 15156792\nSe...,Dr. Abricot,26/10/2018
8,15172164,897653,Loren Moore Née le 05/01/1993\nIPP 15172164\nS...,Dr Strauss,17/01/2001
9,15202908,371185,Patient : Zachery Thompson\nDate de naissance ...,Dr Frank EINSTEIN,14/02/2018


In [23]:
# Ingestion du document dans la base de données
dwh_document.to_sql('dwh_document_raw', \
                con= conn, \
                index=False, \
                if_exists='replace')

15

In [24]:
pd.read_sql_query('select * from dwh_document_raw;', conn)

,IPP,IDDOCUMENT,TEXT,AUTEUR,DATE
0,00230580,321654,Patient : Michelle Personne\nDate de naissance...,Dr Mamy Nova,12/05/2018
1,12558924,090879,Claus Norman Née le 06/10/1999\nIPP 12558924\n...,Dr Hassan,17/10/2011
2,12856116,789456,Alexander Betty Né le 09/11/1990\nIPP 12856116...,Dr. Abricot,26/10/2018
3,12881736,102323,"Paris le 10/02/2015,\nOrdonnance\nIn French po...",Dr Bilingue,10/02/2015
4,12907356,009988,Sabrina Franck Né le 09/07/1980\nIPP 12907356\...,DR Pomme,29/10/2018
5,12943224,976094,"Paris le 12/03/2016,\nOrdonnance\nAvant de voi...",Dr Hu Love,12/03/2016
6,12979092,645329,"Paris le 02/02/2002,\nOrdonnance\nAvant de voi...",Dr Mozza,02/02/2002
7,15156792,111222,Fred Taylor Né le 09/10/1976\nIPP 15156792\nSe...,Dr. Abricot,26/10/2018
8,15172164,897653,Loren Moore Née le 05/01/1993\nIPP 15172164\nS...,Dr Strauss,17/01/2001
9,15202908,371185,Patient : Zachery Thompson\nDate de naissance ...,Dr Frank EINSTEIN,14/02/2018


In [25]:
# Jointure de la table intermédiaire, puis ingestion dans la table DWH_PATIENT_IPPHIST

query = '''
INSERT INTO dwh_document (PATIENT_NUM, ID_DOC_SOURCE, DISPLAYED_TEXT, DOCUMENT_DATE, AUTHOR)
SELECT DISTINCT
    PATIENT_NUM,
    IDDOCUMENT AS ID_DOCUMENT_SOURCE,
    TEXT AS DISPLAYED_TEXT,
    DATE AS DOCUMENT_DATE,
    AUTEUR AS AUTHOR
FROM dwh_document_raw
JOIN DWH_PATIENT_IPPHIST ON IPP = HOSPITAL_PATIENT_ID;'''

In [26]:
conn.execute(query)

In [27]:
pd.read_sql_query('select * from dwh_document', conn)

,DOCUMENT_NUM,PATIENT_NUM,ENCOUNTER_NUM,TITLE,DOCUMENT_ORIGIN_CODE,DOCUMENT_DATE,ID_DOC_SOURCE,DOCUMENT_TYPE,DISPLAYED_TEXT,AUTHOR,UNIT_CODE,UNIT_NUM,DEPARTMENT_NUM,EXTRACTCONTEXT_DONE_FLAG,EXTRACTCONCEPT_DONE_FLAG,ENRGENE_DONE_FLAG,ENRICHTEXT_DONE_FLAG,UPDATE_DATE,UPLOAD_ID
0,1,87,None,None,None,17/10/2011,090879,None,Claus Norman Née le 06/10/1999\nIPP 12558924\n...,Dr Hassan,None,None,None,None,None,None,None,None,None
1,2,145,None,None,None,26/10/2018,789456,None,Alexander Betty Né le 09/11/1990\nIPP 12856116...,Dr. Abricot,None,None,None,None,None,None,None,None,None
2,3,150,None,None,None,10/02/2015,102323,None,"Paris le 10/02/2015,\nOrdonnance\nIn French po...",Dr Bilingue,None,None,None,None,None,None,None,None,None
3,4,155,None,None,None,29/10/2018,009988,None,Sabrina Franck Né le 09/07/1980\nIPP 12907356\...,DR Pomme,None,None,None,None,None,None,None,None,None
4,5,163,None,None,None,12/03/2016,976094,None,"Paris le 12/03/2016,\nOrdonnance\nAvant de voi...",Dr Hu Love,None,None,None,None,None,None,None,None,None
5,6,170,None,None,None,02/02/2002,645329,None,"Paris le 02/02/2002,\nOrdonnance\nAvant de voi...",Dr Mozza,None,None,None,None,None,None,None,None,None
6,7,9,None,None,None,26/10/2018,111222,None,Fred Taylor Né le 09/10/1976\nIPP 15156792\nSe...,Dr. Abricot,None,None,None,None,None,None,None,None,None
7,8,12,None,None,None,17/01/2001,897653,None,Loren Moore Née le 05/01/1993\nIPP 15172164\nS...,Dr Strauss,None,None,None,None,None,None,None,None,None
8,9,18,None,None,None,14/02/2018,371185,None,Patient : Zachery Thompson\nDate de naissance ...,Dr Frank EINSTEIN,None,None,None,None,None,None,None,None,None
9,10,19,None,None,None,12/05/2018,937629,None,Patient : Michelle Personne\nDate de naissance...,None,None,None,None,None,None,None,None,None,None


In [28]:
# Supression de la table intermédiaire
conn.execute('''drop table dwh_document_raw''')

In [29]:
# Lecture des données
pd.read_sql_query('''select * from dwh_document limit 5 ''', conn)

,DOCUMENT_NUM,PATIENT_NUM,ENCOUNTER_NUM,TITLE,DOCUMENT_ORIGIN_CODE,DOCUMENT_DATE,ID_DOC_SOURCE,DOCUMENT_TYPE,DISPLAYED_TEXT,AUTHOR,UNIT_CODE,UNIT_NUM,DEPARTMENT_NUM,EXTRACTCONTEXT_DONE_FLAG,EXTRACTCONCEPT_DONE_FLAG,ENRGENE_DONE_FLAG,ENRICHTEXT_DONE_FLAG,UPDATE_DATE,UPLOAD_ID
0,1,87,None,None,None,17/10/2011,090879,None,Claus Norman Née le 06/10/1999\nIPP 12558924\n...,Dr Hassan,None,None,None,None,None,None,None,None,None
1,2,145,None,None,None,26/10/2018,789456,None,Alexander Betty Né le 09/11/1990\nIPP 12856116...,Dr. Abricot,None,None,None,None,None,None,None,None,None
2,3,150,None,None,None,10/02/2015,102323,None,"Paris le 10/02/2015,\nOrdonnance\nIn French po...",Dr Bilingue,None,None,None,None,None,None,None,None,None
3,4,155,None,None,None,29/10/2018,009988,None,Sabrina Franck Né le 09/07/1980\nIPP 12907356\...,DR Pomme,None,None,None,None,None,None,None,None,None
4,5,163,None,None,None,12/03/2016,976094,None,"Paris le 12/03/2016,\nOrdonnance\nAvant de voi...",Dr Hu Love,None,None,None,None,None,None,None,None,None


In [30]:
# Liste des tables de la base de données
pd.read_sql_query('''select * from sqlite_master''', conn)

,type,name,tbl_name,rootpage,sql
0,table,DWH_PATIENT,DWH_PATIENT,2,CREATE TABLE DWH_PATIENT\n(\n PATIENT_NUM ...
1,table,DWH_PATIENT_IPPHIST,DWH_PATIENT_IPPHIST,3,CREATE TABLE DWH_PATIENT_IPPHIST\n(\n PATIENT...
2,table,DWH_DOCUMENT,DWH_DOCUMENT,4,CREATE TABLE DWH_DOCUMENT\n(\n DOCUMENT_NUM ...


In [31]:
# Fermeture de la connexion
conn.close()